In [1]:
import pandas as pd

url = "https://web.archive.org/web/20010215151137/http://www.tranmererovers.co.uk/managers.htm"

trfc_table = pd.read_html(url, header = 1, flavor = "bs4")[4]
trfc_table = trfc_table.drop(0, axis = 0).drop(trfc_table.index[-3:])

trfc_table.head(3)

,Manager,From,P,W,D,L,F,A,IND,RK
1,Bert Cooke*,27.08.21,578.0,246,116,216,1081,959,105.2,5.0
2,NaN,30.04.35,NaN,42.6,20.1,37.4,1.87,1.66,NaN,NaN
3,Jack Carr,02.07.35,53.0,24,15,14,111,82,118.9,2.0


In [2]:
# Convert From column to date
df = trfc_table.copy()
def convert_date(date_str):
    # Manually add John Aldridge's departure date
    if pd.isna(date_str) or date_str == "current":
        return pd.to_datetime("17 Mar 2001")
    else:
        date = date_str.split(".")
        return pd.to_datetime(f"{date[0]}/{date[1]}/19{date[2]}", dayfirst = True)

        
df["date_from"] = df.From.apply(convert_date)
df["date_to"] = df.date_from.shift(-1)

# drop rows where the Manager column is empty
df = df.dropna(subset = ["Manager"])

df.loc[df.Manager.str.contains("\*{1}", regex = True), "role"] = "Secretary/Manager"
df.loc[df.Manager.str.contains("\*{2}"), "role"] = "Caretaker"
df.role = df.role.fillna("Manager")

df.Manager = df.Manager.str.replace("\*+", "", regex =True).str.replace("\(\d+\)", "", regex =True).str.strip()

df["manager_name"] = df.Manager

trfc_df = df[["manager_name", "date_from", "date_to", "role"]].sort_values("date_from").reset_index(drop = True)

In [3]:
trfc_df.to_csv("./managers-official-pre99.csv", index = False)

In [4]:
sb_table = pd.read_html("https://www.soccerbase.com/teams/team.sd?team_id=2598&teamTabs=managers", flavor = "bs4")[1]

sb_table.head(3)

,Unnamed: 0,FROM,TO
0,Ian Dawes,"20 Mar, 2023",Present
1,Micky Mellon,"01 Jun, 2021","19 Mar, 2023"
2,Keith Hill,"22 Nov, 2020","17 May, 2021"


In [5]:
import datetime as dt

todays_date = dt.date.today()

sb_df = sb_table.copy().rename(columns = {
    "Unnamed: 0": "manager_name",
    "FROM": "date_from",
    "TO": "date_to"
    })
sb_df.loc[sb_df.date_to == "Present", "date_to"] = pd.to_datetime(todays_date)
sb_df.date_from = pd.to_datetime(sb_df.date_from, dayfirst = True, errors = "coerce")
sb_df.date_to = pd.to_datetime(sb_df.date_to, dayfirst = True, errors = "coerce")

sb_df["days_in_charge"] = (sb_df.date_to - sb_df.date_from).dt.days
sb_df.loc[sb_df.days_in_charge < 100, "role"] = "Caretaker"
sb_df.loc[sb_df.days_in_charge >= 100, "role"] = "Manager"
sb_df = sb_df.drop("days_in_charge", axis = 1)

sb_df = sb_df[sb_df.date_from >= trfc_df.date_to.max()]

In [11]:
managers_df = pd.concat([trfc_df, sb_df]).sort_values("date_from").reset_index(drop = True)

In [14]:
managers_df.loc[(managers_df.date_from == "1936-12-01") & (managers_df.manager_name == "Jim Knowles"), "date_to"] = pd.to_datetime("1939-01-18")
managers_df.loc[(managers_df.date_from == "1946-08-31") & (managers_df.manager_name == "Ernie Blackburn"), "date_from"] = pd.to_datetime("1946-09-16")
managers_df.loc[(managers_df.date_from == "1955-12-13") & (managers_df.manager_name == "Noel Kelly"), "date_to"] = pd.to_datetime("1956-11-12")
managers_df.loc[(managers_df.date_from == "1961-01-07") & (managers_df.manager_name == "Walter Galbraith"), "date_to"] = pd.to_datetime("1961-12-08")
managers_df.loc[(managers_df.date_from == "1975-04-04") & (managers_df.manager_name == "John King"), "date_to"] = pd.to_datetime("1980-09-05")


additions = [
    {
        "manager_name": "Jimmy Morton",
        "date_from": pd.to_datetime("1939-01-18"),
        "date_to": pd.to_datetime("1942-02-28"),
        "role": "Manager"
    },
    {
        "manager_name": "Bill Ridding",
        "date_from": pd.to_datetime("1942-03-02"),
        "date_to": pd.to_datetime("1946-07-19"),
        "role": "Manager"
    },
    {
        "manager_name": "Tommy Jones",
        "date_from": pd.to_datetime("1946-08-10"),
        "date_to": pd.to_datetime("1946-09-15"),
        "role": "Manager"
    },
    {
        "manager_name": "Ernie Blackburn",
        "date_from": pd.to_datetime("1956-11-12"),
        "date_to": pd.to_datetime("1957-10-04"),
        "role": "Manager"
    },
    {
        "manager_name": "No manager",
        "date_from": pd.to_datetime("1961-12-09"),
        "date_to": pd.to_datetime("1961-12-13"),
        "role": "Manager"
    },
    {
        "manager_name": "No manager",
        "date_from": pd.to_datetime("1980-09-06"),
        "date_to": pd.to_datetime("1980-09-30"),
        "role": "Manager"
    },
    {
        "manager_name": "Jason McAteer",
        "date_from": pd.to_datetime("2006-05-05"),
        "date_to": pd.to_datetime("2006-05-07"),
        "role": "Manager"
    },
    {
        "manager_name": "Ian Dawes",
        "date_from": pd.to_datetime("2021-05-17"),
        "date_to": pd.to_datetime("2021-05-21"),
        "role": "Manager"
    }
]
additions = pd.DataFrame(additions)

dawes_appointment = "2023-05-04"
managers_df.loc[(managers_df.date_from == "2023-03-20") & (managers_df.manager_name == "Ian Dawes"), "date_to"] = pd.to_datetime(dawes_appointment)
managers_df.loc[(managers_df.date_from == "2023-03-20") & (managers_df.manager_name == "Ian Dawes"), "role"] = "Caretaker"

dawes_perm = pd.DataFrame({
    "manager_name": "Ian Dawes",
    "date_from": pd.to_datetime(dawes_appointment),
    "date_to": pd.to_datetime(todays_date),
    "role": "Manager"
}, index = [0])

final_df = pd.concat([managers_df, dawes_perm, additions]).sort_values("date_from").reset_index(drop = True)
final_df

,manager_name,date_from,date_to,role
0,Bert Cooke,1921-08-27,1935-04-30,Secretary/Manager
1,Jack Carr,1935-07-02,1936-10-22,Manager
2,No manager,1936-10-23,1936-11-30,Manager
3,Jim Knowles,1936-12-01,1939-01-18,Secretary/Manager
4,Jimmy Morton,1939-01-18,1942-02-28,Manager
5,Bill Ridding,1942-03-02,1946-07-19,Manager
6,Tommy Jones,1946-08-10,1946-09-15,Manager
7,Ernie Blackburn,1946-09-16,1955-12-12,Secretary/Manager
8,Noel Kelly,1955-12-13,1956-11-12,Manager
9,Ernie Blackburn,1956-11-12,1957-10-04,Manager


In [15]:
final_df.to_csv("../output/managers.csv", index = False)